# Preparando os dados do IPTU de 2022

Para o primeiro experimento vamos preparar os dados do IPTU de 2022, e posteriormente validar com as informações apresentadas no DashBoard

In [ ]:
import vaex
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
# import pygeos

In [ ]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras = gdf_quadras.dissolve(['qd_setor', 'qd_fiscal']).reset_index()
gdf_quadras_centroid = gdf_quadras.copy()
gdf_quadras_centroid.geometry = gdf_quadras_centroid.representative_point()

In [ ]:
gdf_quadras.shape, gdf_quadras_centroid.shape

In [ ]:
gdf_quadras_distritos = gdf_quadras_centroid.sjoin(gdf_distritos, how='left', predicate='intersects')
gdf_quadras_distritos.drop('index_right', axis=1, inplace=True)

In [ ]:
gdf_quadras.shape, gdf_quadras_distritos.shape

In [ ]:
gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017')
gdf_od_sp = gdf_od[gdf_od.od_municip == '36']
gdf_od_sp.set_crs(epsg=31983, inplace=True)
gdf_quadras_distritos_od = gdf_quadras_distritos.sjoin(gdf_od_sp, how='left', predicate='intersects')

df_quadras_distritos_od = gdf_quadras_distritos_od.loc[:, ['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub', 'ds_subpref', 'od_id', 'od_nome']]
df_quadras_distritos_od.loc[:, ['sq']] = df_quadras_distritos_od.qd_setor + df_quadras_distritos_od.qd_fiscal
# df_quadras_distritos_od.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [ ]:
gdf_quadras.shape, df_quadras_distritos_od.shape

In [ ]:
gdf_ponderacao_censo = gpd.read_file('data/areas-ponderacao-censo.gpkg', layer='areas-ponderacao-censo-2010')
df_quadras_censo =  gdf_quadras_centroid.sjoin(gdf_ponderacao_censo, how='left', predicate='intersects')

In [ ]:
df_quadras_distritos_od.shape, df_quadras_censo[['COD_AED', 'COD_AED_S']].shape

In [ ]:
df_quadras_distritos_od = pd.concat(
    [df_quadras_distritos_od,
    df_quadras_censo[['COD_AED', 'COD_AED_S']]],
    axis=1
)

In [ ]:
df_quadras_distritos_od.columns

In [ ]:
gdf_od_sp = gdf_od_sp.to_file('data/OD-sp.gpkg', driver='GPKG')

In [ ]:
df_quadras_distritos_od

In [ ]:
## TODO
# Verificar essa inconsistencia
gdf_quadras_distritos[gdf_quadras_distritos.ds_codigo.isna()]

In [ ]:
dfs = []

for i in range(1995,2023):

    print(f'Processando {i}')
    
    # df = vaex.open(f'data/IPTU_{i}/IPTU_{i}.hdf5')
    df = vaex.open(f'data//IPTU-HDF5/IPTU_{i}/IPTU_{i}.hdf5')

    df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
    df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
    df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
    df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
    df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
    df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
    df['sq'] = df.setor + df.quadra
    df['sqlc'] = df.sqlc
    df_sqlc_ac = df.groupby('sqlc', agg={'area_contruida_total': vaex.agg.sum('AREA CONSTRUIDA')})
    df = df.join(df_sqlc_ac, on='sqlc')
    df['fracao_ideal'] = df.func.where(df['FRACAO IDEAL'] == 0.,  
                                                df['AREA CONSTRUIDA'] / df['area_contruida_total'],
                                                df['FRACAO IDEAL'])
    df['ca'] = (df['AREA CONSTRUIDA'] / df['fracao_ideal']) / df['AREA DO TERRENO']
    df['to'] = df['AREA OCUPADA'] / df['AREA DO TERRENO']

    df['valor_terreno'] = df['fracao_ideal'] * df['VALOR DO M2 DO TERRENO'] * df['AREA DO TERRENO']
    df['valor_construcao'] = df['VALOR DO M2 DE CONSTRUCAO'] * df['AREA CONSTRUIDA']

    dfs.append(df)
    # break


In [ ]:
df_iptu = vaex.concat(dfs)

In [ ]:
df_iptu.shape

In [ ]:
## Parece que para alguns lotes condominiais o terreno está proporcional à fracao ideal
df_iptu[df_iptu.ca > 100]['AREA CONSTRUIDA', 'fracao_ideal', 'AREA DO TERRENO', 'area_contruida_total', 'ANO DO EXERCICIO']

In [ ]:
df_iptu[df_iptu['NUMERO DO CONDOMINIO'] == '00-0'].shape

In [ ]:
df_iptu.shape

In [ ]:
df_quadras_distritos_od['sqod'] = df_quadras_distritos_od.sq + df_quadras_distritos_od.od_id.str.pad(width=3, side='left', fillchar='0')

In [ ]:
df_quadras_distritos_od.shape

In [ ]:
df_quadras_distritos_od

In [ ]:
df_quadras_distritos_od = df_quadras_distritos_od.drop_duplicates(subset=['sq'])

In [ ]:
df_quadras_distritos_od.shape, df_iptu.shape

In [ ]:
df_iptu = df_iptu.join(vaex.from_pandas(df_quadras_distritos_od), on='sq', how='left',  
            allow_duplication=False, inplace=False)

In [ ]:
df_quadras_distritos_od.shape, df_iptu.shape

## Adicionando informações de Uso

In [ ]:
df_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
df_usos_2022

In [ ]:
df_usos = df_iptu.groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()

In [ ]:
df_usos.fillna('', inplace=True)

In [ ]:
results, ratios = [], []

for i, row in df_usos.iterrows():

    similaridade = df_usos_2022['TIPO DE USO DO IMOVEL'].apply(lambda x: SequenceMatcher(None, row['TIPO DE USO DO IMOVEL'], x).ratio())
    
    results.append(df_usos_2022.iloc[similaridade.idxmax()]['TIPO DE USO DO IMOVEL'])

    ratios.append(similaridade.max())

df_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [ ]:
df_usos

In [ ]:
residencial = ['residencial', 'residência', 'apartamento', 'cortiço']
comercial = ['comércio', 'comercial', 'posto', 'loja']
servicos =  ['hotel', 'radioemissora', 'flat', 'serviço', 'clube', 'oficina', 'hospital', 'templo', 'escritórios', 'consultórios', 'escritório', 'consultório', 'asilo', 'creche', 'prédio de garagens', 'escola']
industrial = ['Indústria', 'armazéns']

In [ ]:
class_residencial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(residencial)), case=False)
class_comercial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(comercial)), case=False)
class_servicos = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(servicos)), case=False)
class_industrial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(industrial)), case=False)

df_usos.loc[class_residencial, ['classificacao_abrangente']] = 'Residencial'
df_usos.loc[class_comercial, ['classificacao_abrangente']] = 'Comercial'
df_usos.loc[class_servicos, ['classificacao_abrangente']] = 'Serviços'
df_usos.loc[class_industrial, ['classificacao_abrangente']] = 'Industrial'

df_usos.loc[df_usos.classificacao_abrangente.isna(), ['classificacao_abrangente']] = 'Outros'

In [ ]:
# df_iptu['TIPO DE USO DO IMOVEL']
df_usos = vaex.from_pandas(df_usos.loc[df_usos['TIPO DE USO DO IMOVEL'] != '', ['TIPO DE USO DO IMOVEL', 'classificacao_abrangente']])
df_iptu = df_iptu.join(df_usos, on='TIPO DE USO DO IMOVEL')

In [ ]:
df_iptu[['classificacao_abrangente']]

In [ ]:
df_iptu.shape

In [ ]:
df_padroes_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos_2022.loc[df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'] == 'TERRENO', 'TIPO DE PADRAO DA CONSTRUCAO'] = 'Terreno'
df_padroes_usos_2022

In [ ]:
df_padroes_usos = df_iptu.groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos.fillna('', inplace=True)
df_padroes_usos

In [ ]:
results, ratios = [], []

for i, row in df_padroes_usos.iterrows():

    similaridade = df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'].apply(lambda x: SequenceMatcher(None, row['TIPO DE PADRAO DA CONSTRUCAO'], x).ratio())
    
    results.append(df_padroes_usos_2022.iloc[similaridade.idxmax()]['TIPO DE PADRAO DA CONSTRUCAO'])

    ratios.append(similaridade.max())

df_padroes_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_padroes_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [ ]:
# df_padroes_usos
df_padroes_usos['tipo_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[0] if x[0][-1] != ' ' else x[0][:-1])
df_padroes_usos['padrao_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[-1][-1] if x[-1][-3] == 'o' else '')

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'A') |
                    (df_padroes_usos.padrao_de_uso == 'B'),
                    'classe_de_uso'] = 'Baixo'

df_padroes_usos.loc[df_padroes_usos.padrao_de_uso == 'C',
                    'classe_de_uso'] = 'Médio'

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'D') |
                    (df_padroes_usos.padrao_de_uso == 'E') |
                    (df_padroes_usos.padrao_de_uso == 'F'),
                    'classe_de_uso'] = 'Alto'

df_padroes_usos.fillna('', inplace=True)


In [ ]:
df_padroes_usos

In [ ]:
df_padroes_usos.rename(columns={'titulo_padronizado':'padrao_construcao'}, inplace=True)
df_padroes_usos = vaex.from_pandas(df_padroes_usos.loc[df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'] != '', ['TIPO DE PADRAO DA CONSTRUCAO', 'padrao_construcao', 'tipo_de_uso', 'padrao_de_uso', 'classe_de_uso']])
df_iptu = df_iptu.join(df_padroes_usos, on='TIPO DE PADRAO DA CONSTRUCAO')

In [ ]:
df_iptu

## Processando as totalizacoes

In [ ]:
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.to_crs(epsg=4674, inplace=True)

In [ ]:
df_iptu['sq'] = df_iptu.qd_setor + df_iptu.qd_fiscal

In [ ]:
# df_iptu.sq

In [ ]:
# df_iptu.fillna(value='0', column_names=['sq'], inplace=True)
# df_iptu['sq'] = df_iptu['sq'].astype('int')
# df_iptu.categorize('sq', inplace=True)

In [ ]:
# df_iptu.head(1)
df_iptu.fillna(value='0', column_names=['ds_codigo'], inplace=True)
df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu.categorize('distrito', inplace=True)
# df_iptu.head(1)

In [ ]:
# df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu['ano'] = df_iptu['ANO DO EXERCICIO'].astype('int')
df_iptu.categorize('ano', inplace=True)

In [ ]:
df_iptu.fillna(value='0', column_names=['od_id'], inplace=True)
df_iptu['od'] = df_iptu['od_id'].astype('int')
df_iptu.categorize('od', inplace=True)

In [ ]:
df_iptu.shape

In [ ]:
df_iptu.fillna(value='0', column_names=['ds_cd_sub'], inplace=True)
df_iptu['subprefeitura'] = df_iptu['ds_cd_sub'].astype('int')
df_iptu.categorize('subprefeitura', inplace=True)

In [ ]:
df_iptu.fillna(value='0', column_names=['COD_AED_S'], inplace=True)
df_iptu['censo'] = df_iptu['COD_AED_S'].astype('int')
df_iptu.categorize('censo', inplace=True)


In [ ]:
# df_iptu_grouped_distrito = \
def agrupamento(campo):
    return df_iptu.groupby([campo, 'ano'],agg={
                        'Quantidade de Unidades': 'count', 
                        'Quantidade de Unidades Condominiais': vaex.agg.count('sqlc',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho Médio da Unidade Condominial': vaex.agg.mean('AREA CONSTRUIDA', 
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho médio dos Terrenos': vaex.agg.mean('AREA DO TERRENO'),
                        'Área Total dos terrenos-lotes': vaex.agg.sum('AREA DO TERRENO'),
                        'Área Total Ocupada': vaex.agg.sum('AREA OCUPADA'),
                        'Área Total Construída':vaex.agg.sum('AREA CONSTRUIDA'),
                        'Valor Total dos Terrenos': vaex.agg.sum('valor_terreno'),
                        'Valor Total das Construções': vaex.agg.sum('valor_construcao'),
                        'CA médio': vaex.agg.mean('ca'),
                        'TO médio': vaex.agg.mean('to'),
                        'CA médio em lotes condominiais': vaex.agg.mean('ca',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'TO médio em lotes condominiais': vaex.agg.mean('to',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] != '00-0'),
                        'CA médio em lotes não condominiais': vaex.agg.mean('ca',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] == '00-0'),
                        'TO médio em lotes não condominiais': vaex.agg.mean('to',
                                                                        selection=df_iptu['NUMERO DO CONDOMINIO'] == '00-0'),
                        'Comprimento Médio da Testada': vaex.agg.mean('TESTADA PARA CALCULO'),
                        'Número médio de Pavimentos': vaex.agg.mean('QUANTIDADE DE PAVIMENTOS'),
                        'Fator de obsolecência médio': vaex.agg.mean('FATOR DE OBSOLESCENCIA'),
                        'Residencial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Residencial'),
                        'Comercial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Comercial'),
                        'Serviços': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Serviços'),
                        'Industrial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Industrial'),
                        'Outros': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df_iptu['classificacao_abrangente'] == 'Outros'),
                        # 'Padrão de Construção A - Baixo': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'A'),
                        # 'Padrão de Construção B - Baixo': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'B'),                                                                                                                                                                
                        # 'Padrão de Construção C - Médio': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'C'),
                        # 'Padrão de Construção D - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'D'),
                        # 'Padrão de Construção E - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'E'),
                        # 'Padrão de Construção F - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                        #                                             selection=df_iptu['padrao_de_uso'] == 'F'),                                                                                                                                                                                                            
    })

In [ ]:
campos = ['distrito', 'subprefeitura', 'od', 'censo', 'sq', 'sqlc']

In [ ]:
dfs = []

for campo in campos:
    print(campo)
    df = agrupamento(campo)
    for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
        df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
    df.export_hdf5(f'data/IPTU-1995-2022-agrupados-por-{campo}.hdf5')
    dfs.append(df)

In [ ]:
# df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
# df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
# df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

df_iptu_distrito, df_iptu_subprefeitura, df_iptu_od, df_iptu_censo, df_iptu_sq, df_iptu_sqlc = dfs

In [ ]:
f"{format(df_iptu_distrito['Quantidade de Unidades'].sum(), ',d').replace(',', '.')} seila"

In [ ]:
df_iptu_sqlc.shape

In [ ]:
df_iptu_subprefeitura["CA médio"].max()

In [ ]:
df_iptu_od["CA médio"].max()

In [ ]:
df_iptu.sqlc